<a href="https://colab.research.google.com/github/skovz99/kNN_Pruning/blob/main/kNN_Pruning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
from sqlalchemy.sql.functions import count
import numpy as np
import pandas as pd
import math
from sklearn.neighbors import NearestNeighbors
import warnings
warnings.filterwarnings("ignore")

def neighbors(training_X, nbrs):
  x = NearestNeighbors(n_neighbors=nbrs, algorithm='ball_tree').fit(training_X)
  distances, indices = x.kneighbors(training_X)
  first_df = pd.DataFrame(indices[:, 1:])
  new_columns = [f'Neighbor {x + 1}' for x in range(nbrs-1)]
  first_df.columns = new_columns
  return first_df

def create_list(resulting_indices):
  combined_series = pd.Series()
  for column_name in resulting_indices.columns:
    combined_series = combined_series.append(resulting_indices[column_name])
  combined_list = combined_series.tolist()
  return combined_list

def unique_values(listing, number_guide):
  number = np.array(number_guide)
  counts = []
  for num in number:
    save = listing.count(num)
    counts.append(save)
  return counts

def percentile(counting, k):
  array = np.array(counting)
  percentile = np.percentile(array, k)
  return percentile

def prune(percentage, counting):
  count_df = pd.DataFrame()
  count_df['Counting'] = counting
  count_df['Counting'] = np.where(count_df['Counting'] <= percentage, np.nan, count_df['Counting'])
  return count_df

In [24]:
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])

first = neighbors(X, nbrs = 4)
second = create_list(first)
third = unique_values(second, range(len(X)))
fourth = percentile(third, k = 10) # k value represents a percentile between 0 and 100 that you choose, recommended between 0 and 15
fifth = prune(fourth, third)

# what to do after running the five functions
sixth = pd.DataFrame(X)
sixth = sixth.rename(columns={0: "X", 1: "Y"}) # if you already have varaible names you can skip this step and just concat sixth and seventh
seventh = pd.DataFrame(pd.concat([sixth, fifth], axis = 1))

# drop the NaN rows from the dataframe and all rows left will have the variables that are most frequently used
seventh = seventh.dropna()
